In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import transforms
import data
from model.unet import U_Net
from model.simple_unet import Simple_Unet
import matplotlib.pyplot as plt
import torch
from torchsummary import summary
import glob
from sklearn.model_selection import train_test_split

/Users/maochenhui/miniforge3/envs/DL/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
model = Simple_Unet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, (3, 2302, 1632))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 9, 1152, 832]             252
              ReLU-2         [-1, 9, 1152, 832]               0
       BatchNorm2d-3         [-1, 9, 1152, 832]              18
        conv_block-4         [-1, 9, 1152, 832]               0
            Conv2d-5         [-1, 12, 576, 416]             984
              ReLU-6         [-1, 12, 576, 416]               0
       BatchNorm2d-7         [-1, 12, 576, 416]              24
        conv_block-8         [-1, 12, 576, 416]               0
            Conv2d-9         [-1, 12, 288, 208]           1,308
             ReLU-10         [-1, 12, 288, 208]               0
      BatchNorm2d-11         [-1, 12, 288, 208]              24
       conv_block-12         [-1, 12, 288, 208]               0
         Upsample-13         [-1, 12, 576, 416]               0
  ConvTranspose2d-14          [-1, 9, 5

In [3]:
SCANS_DIR = "input/scans/scans/"
TRUTH_DIR = "input/ground-truth-pixel/ground-truth-pixel/"
scan_files = glob.glob(SCANS_DIR+'*.png')
scan_files = sorted(scan_files)
# scan_files = list(filter(lambda x:"(" not in x, scan_files))
scan_files_dev = scan_files[:400]

scan_files_test = scan_files[400:]
truth_files = glob.glob(TRUTH_DIR+'*.png')
truth_files = sorted(truth_files)
# truth_files = list(filter(lambda x:"(" not in x, truth_files))

scan_files_train, scan_files_val, truth_files_train, truth_files_val = train_test_split(scan_files_dev, truth_files, test_size=0.3)

In [4]:
from torch.utils.data import DataLoader
train_data = data.CustomDataset(scans = scan_files_train, ground_truth=truth_files_train, transforms=transforms.default_transform["train"])
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)

val_data = data.CustomDataset(scans=scan_files_val, ground_truth=truth_files_val, transforms=transforms.default_transform["val"])
val_loader = DataLoader(val_data, batch_size=1)

In [6]:
from train import Trainer

train = (scan_files_train, truth_files_train)
val = (scan_files_val, truth_files_val)
trainer = Trainer(train, val, model, batch_size=1, epochs=100, lr=1e-2, step_size=100)

In [10]:
import sys
import time

msg = ''
for i in range(100):
    msg += msg.join('.')
    a = "耐心等待" + '{}'.format(msg)
    if i % 10 == 0:     # 到第几次重头开始
        msg = ''
    print("%d\r" % i, end='', flush=True)
    time.sleep(0.1)